<a href="https://colab.research.google.com/github/yeongju98/2022_1_DA/blob/main/CV_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.8.0

In [39]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
# TODO: Change Models You want https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet
from tensorflow.keras.applications import ResNet50V2

from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import numpy as np

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.8.0


## **CIFAR 10 Datast 데이터셋 다운로드하고 준비하기**




In [40]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  # TODO: Change Models You want to change
  output_ims = tf.keras.applications.resnet_v2.preprocess_input(input_images)
  return output_ims

In [41]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

In [42]:
train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

In [43]:
train_X.shape

(50000, 32, 32, 3)

In [44]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  #Change Models Here too!!!
  
  feature_extractor = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='Adam', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

94683136/94668760 [==============================] - 1s 0us/step
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d_3 (UpSampling  (None, 224, 224, 3)      0         
 2D)                                                             
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_3 (Flatten)         (None, 2048)              0         
                                                            

In [45]:
EPOCHS = 3
history = model.fit(train_X, training_labels, epochs=EPOCHS, validation_data = (valid_X, validation_labels), batch_size=64)

Epoch 1/3
782/782 [==============================] - 540s 612ms/step - loss: 0.7785 - accuracy: 0.7355 - val_loss: 0.8183 - val_accuracy: 0.7271
Epoch 2/3
782/782 [==============================] - 488s 624ms/step - loss: 0.4448 - accuracy: 0.8503 - val_loss: 0.6439 - val_accuracy: 0.7921
Epoch 3/3
782/782 [==============================] - 487s 623ms/step - loss: 0.3190 - accuracy: 0.8909 - val_loss: 0.4812 - val_accuracy: 0.8402


In [46]:
loss, accuracy = model.evaluate(valid_X, validation_labels, batch_size=64)

157/157 [==============================] - 28s 176ms/step - loss: 0.4812 - accuracy: 0.8402
